In [69]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.gridspec as gridspec
import os

In [55]:
n_input = 784
n_h1 = 128
n_h2 = 1
n_z = 100
mb_size = 128
#Z_dim = 100

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None, 784])

D_weights= \
{
    'W1': tf.Variable(xavier_init([n_input, n_h1])),
    'W2': tf.Variable(xavier_init([n_h1, n_h2]))
}

D_biases = \
{
    'b1': tf.Variable(tf.zeros(shape=[n_h1])),
    'b2': tf.Variable(tf.zeros(shape=[n_h2]))
}

Z = tf.placeholder(tf.float32, shape=[None, n_z])

G_weights = \
{
    'W1': tf.Variable(xavier_init([n_z, n_h1])),
    'W2': tf.Variable(xavier_init([n_h1, n_input]))
}

G_biases = \
{
    'b1': tf.Variable(tf.zeros(shape=[n_h1])),
    'b2': tf.Variable(tf.zeros(shape=[n_input]))
}

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [57]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def descriminator(x):
    h1 = tf.nn.relu(tf.matmul(x, D_weights['W1']) + D_biases['b1'])
    D_logit = tf.matmul(h1, D_weights['W2']) + D_biases['b2']
    D_prob = tf.nn.sigmoid(D_logit)
    return D_logit, D_prob

def generator(z):
    h1 = tf.nn.relu(tf.matmul(z, G_weights['W1']) + G_biases['b1'])
    G_logit = tf.matmul(h1, G_weights['W2']) + G_biases['b2']
    G_prob = tf.nn.sigmoid(G_logit)    
    return G_prob

In [63]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [56]:
G = generator(Z)

D_log_fake, D_prob_fake = descriminator(G)
D_log_real, D_prob_real = descriminator(X)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_log_real, labels=tf.ones_like(D_log_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_log_fake, labels=tf.ones_like(D_log_fake)))

D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_log_fake, labels=tf.ones_like(D_log_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=[D_weights['W1'], D_weights['W2'], D_biases['b1'], D_biases['b2']])
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=[G_weights['W1'], G_weights['W2'], G_biases['b1'], G_biases['b2']])

In [ ]:
if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for it in range(1000000):
    if it % 1000 == 0:
        samples = sess.run(G, feed_dict={Z: sample_Z(16, n_z)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
        
    X_mb, _ = mnist.train.next_batch(mb_size)
    
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, n_z)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, n_z)})
    
    if it % 1000 == 0:
        print('Iter: {}'.format(i))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 1
D loss: 0.8822
G_loss: 0.02006
()
Iter: 2
D loss: 5.383e-06
G_loss: 8.706e-24
()
Iter: 3
D loss: 5.861e-07
G_loss: 1.323e-26
()
